# Verifying Seebuoy data for NDBC on Station 44150

In [29]:
pip install seebuoy

Note: you may need to restart the kernel to use updated packages.


In [30]:
from seebuoy import NDBC
import pandas as pd
import numpy as np

ndbc = NDBC()

# Information on NDBC's ~1800 buoys and gliders
wave_df = ndbc.stations()

# list all available data for all buoys
df_data = ndbc.available_data()

# Get info on La Jave Bank (42.500N 64.02W) 
# SEEBUOY DATA
station_id = "44065"
see_buoy = ndbc.get_data(station_id)
see_buoy

,wind_direction,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,air_temp,water_temp,dewpoint,visibility,pressure_tendency,tide
date,,,,,,,,,,,,,,
2023-12-22 00:00:00,NaN,11.0,13.0,NaN,NaN,NaN,NaN,1028.4,NaN,10.0,NaN,NaN,0.6,NaN
2023-12-22 00:10:00,NaN,11.0,13.0,NaN,NaN,NaN,NaN,1028.4,NaN,10.0,NaN,NaN,NaN,NaN
2023-12-22 00:20:00,NaN,11.0,14.0,1.2,11.0,4.3,119.0,1028.4,NaN,10.0,NaN,NaN,NaN,NaN
2023-12-22 00:30:00,NaN,11.0,14.0,NaN,NaN,NaN,NaN,1028.5,NaN,10.0,NaN,NaN,NaN,NaN
2023-12-22 00:40:00,NaN,11.0,14.0,NaN,NaN,NaN,NaN,1028.7,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-05 20:40:00,NaN,6.0,7.0,NaN,NaN,NaN,NaN,1011.9,NaN,6.0,NaN,NaN,NaN,NaN
2024-02-05 20:50:00,NaN,5.0,7.0,1.1,11.0,6.1,103.0,1011.8,NaN,6.0,NaN,NaN,NaN,NaN
2024-02-05 21:00:00,NaN,6.0,7.0,NaN,NaN,NaN,NaN,1011.9,NaN,6.0,NaN,NaN,-0.3,NaN


In [31]:
# NDBC TXT
# Define the width of each column based on the spacing in your data
column_widths = [4, 3, 3, 3, 3, 5, 5, 5, 6, 6, 5, 5, 7, 7, 5, 5, 6, 4, 6]

# Read the fixed-width file into a Pandas DataFrame
ndbc = pd.read_fwf("44065.txt", widths=column_widths, header=[0])
ndbc = ndbc.drop(labels=0, axis=0)
ndbc

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,PTDY,TIDE
1,2024,02,05,21,30,MM,6.0,8.0,MM,MM,MM,MM,1012.0,MM,6.0,MM,MM,MM,MM
2,2024,02,05,21,20,MM,6.0,7.0,1.0,11,5.8,108,1012.0,MM,6.0,MM,MM,MM,MM
3,2024,02,05,21,10,MM,6.0,7.0,1.0,MM,5.8,108,1012.0,MM,6.0,MM,MM,MM,MM
4,2024,02,05,21,00,MM,6.0,7.0,MM,MM,MM,MM,1011.9,MM,6.0,MM,MM,-0.3,MM
5,2024,02,05,20,50,MM,5.0,7.0,1.1,11,6.1,103,1011.8,MM,6.0,MM,MM,MM,MM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,2023,12,22,00,40,MM,11.0,14.0,MM,MM,MM,MM,1028.7,MM,10.0,MM,MM,MM,MM
6373,2023,12,22,00,30,MM,11.0,14.0,MM,MM,MM,MM,1028.5,MM,10.0,MM,MM,MM,MM
6374,2023,12,22,00,20,MM,11.0,14.0,1.2,11,4.3,119,1028.4,MM,10.0,MM,MM,MM,MM
6375,2023,12,22,00,10,MM,11.0,13.0,MM,MM,MM,MM,1028.4,MM,10.0,MM,MM,MM,MM


In [32]:
# consolidate columns into one date column
ndbc['date'] = ndbc.apply(lambda row: f"{int(row['#YY'])}-{int(row['MM']):02d}-{int(row['DD']):02d} {int(row['hh']):02d}:{int(row['mm']):02d}:00", axis=1)

ndbc = ndbc.drop(['#YY', 'MM', 'DD', 'hh', 'mm'], axis=1)

# replace columns with see_buoy column names
ndbc.rename(columns={'WDIR': 'wind_direction', 'WSPD': 'wind_speed', 'GST':'wind_gust', 'WVHT':'wave_height', 'DPD':'dominant_period',
                        'APD':'average_period', 'MWD':'mean_wave_direction', 'PRES':'pressure', 'ATMP':'air_temp', 'WTMP':'water_temp', 'DEWP':'dewpoint',
                        'VIS':'visibility', 'PTDY':'pressure_tendency', 'TIDE':'tide'}, inplace=True)

# replace MM with NaN
ndbc.replace('MM', np.nan, inplace=True)
ndbc

,wind_direction,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,air_temp,water_temp,dewpoint,visibility,pressure_tendency,tide,date
1,NaN,6.0,8.0,NaN,NaN,NaN,NaN,1012.0,NaN,6.0,NaN,NaN,NaN,NaN,2024-02-05 21:30:00
2,NaN,6.0,7.0,1.0,11,5.8,108,1012.0,NaN,6.0,NaN,NaN,NaN,NaN,2024-02-05 21:20:00
3,NaN,6.0,7.0,1.0,NaN,5.8,108,1012.0,NaN,6.0,NaN,NaN,NaN,NaN,2024-02-05 21:10:00
4,NaN,6.0,7.0,NaN,NaN,NaN,NaN,1011.9,NaN,6.0,NaN,NaN,-0.3,NaN,2024-02-05 21:00:00
5,NaN,5.0,7.0,1.1,11,6.1,103,1011.8,NaN,6.0,NaN,NaN,NaN,NaN,2024-02-05 20:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6372,NaN,11.0,14.0,NaN,NaN,NaN,NaN,1028.7,NaN,10.0,NaN,NaN,NaN,NaN,2023-12-22 00:40:00
6373,NaN,11.0,14.0,NaN,NaN,NaN,NaN,1028.5,NaN,10.0,NaN,NaN,NaN,NaN,2023-12-22 00:30:00
6374,NaN,11.0,14.0,1.2,11,4.3,119,1028.4,NaN,10.0,NaN,NaN,NaN,NaN,2023-12-22 00:20:00
6375,NaN,11.0,13.0,NaN,NaN,NaN,NaN,1028.4,NaN,10.0,NaN,NaN,NaN,NaN,2023-12-22 00:10:00


In [38]:
# verify the data is correct

see_buoy['wind_direction'].equals(ndbc['wind_direction'])  # Returns True



False